In [1]:
import numpy as np
import sys
sys.path.append('../../')
sys.path.append('../../pyfos/')
from pyfos.models.feedforward import build_feedforward
from pyfos.models.var_feedforward import build_var_feedforward
from pyfos.generic.optimizer import gd_optimizer
from pyfos.generic.regularizers import gaus_prior
from functools import partial 
from pyfos.tc.simple import  simple_tc
from pyfos.tc.cv import  cv_tc
from pyfos.hyperoptimizers.random_search import random_optimize
from pyfos.hyperoptimizers.greed_optimize import greed_optimize
from pyfos.hyperoptimizers.drmad_optimize import drmad_optimize
from pyfos.hyperoptimizers.hoag_optimize import hoag_optimize

import theano
%matplotlib inline
import matplotlib.pylab as plt
import random
import theano.tensor as T

In [8]:
X_train, X_test = np.load('../../../data/mnist_train_x.npy'), np.load('../../../data/mnist_test_x.npy')
Y_train, Y_test = np.load('../../../data/mnist_train_y.npy'), np.load('../../../data/mnist_test_y.npy')
hid = 300
param_num = X_train.shape[1] * hid + hid + hid*10 + 10
np.set_printoptions(threshold=5)
lr = theano.shared(10**(-5))

init2 = [np.sqrt(2.0/X_train.shape[1]), np.sqrt(2.0/hid)]
alphas = theano.shared(np.array([1.0, 1.0]))
real_alphas = T.concatenate([T.repeat(alphas[0],  X_train.shape[1] * hid + hid)   , T.repeat(alphas[1],  hid*10 + 10) ])
optimizer = partial(gd_optimizer, learning_rate=lr)
model_build = partial(build_feedforward,  structure = [X_train.shape[1],hid, 10], 
                      init_sigmas=init2, use_softmax=True, nonlinearity=lambda x:T.nnet.relu(x), 
                      log_alphas =real_alphas, bias=True)

init2 = [np.sqrt(2.0/X_train.shape[1]), np.sqrt(2.0/hid)]

var_alphas = theano.shared(np.array([1.0]*param_num))
var_model_build = partial(build_var_feedforward,   param_pool_size=100, structure = [X_train.shape[1],hid, 10],
                          use_softmax=True,
                          init_sigmas=init2, nonlinearity=lambda x:T.nnet.relu(x), log_alphas =var_alphas, bias=True)

#for test
alphas.set_value(np.array([12., 1.0]))
print alphas.eval()
print real_alphas.eval()

[ 12.   1.]
[ 12.  12.  12. ...,   1.   1.   1.]


In [ ]:
np.max(abs(np.random.randn(29000)*0.2))

In [7]:
#Greed
cnter = 1
ranges = []
for k in xrange(-10, -6):
    for j in xrange(1,5):
        ranges.append(10**(k)*(j*25))
ranges.sort(reverse=True)
print ranges
hyp_lr_range = [ranges[cnter]] 

print 'calbirate: alpha'
found = False
for h in [  [np.array([-6.0,-6.0])], 
            
             [np.array([6.0, 6.0])]]:
    
    print h
    while True:

            
            
            alphas.set_value(h[0])



            score = greed_optimize(partial(cv_tc, k =4,  batch_size=100), model_build, optimizer,10,25, 12500,  X_train, Y_train,  [alphas] , 
                   lr=hyp_lr_range, verbose=500
            ).history[-1][1]
            if lr.eval()>1 or np.isnan(lr.eval()) or np.isnan(np.sum(alphas.eval())) or \
             max(abs(alphas.eval()))>15 or \
            np.isnan(score) or np.isinf(score):
                print 'BAD'
                cnter+=1
                hyp_lr_range = [ranges[cnter]] 
                print 'new range', hyp_lr_range
            else:
                
                break
            
print 'final', hyp_lr_range            
        
        
        


[9.999999999999999e-06, 7.499999999999999e-06, 4.9999999999999996e-06, 2.4999999999999998e-06, 1e-06, 7.5e-07, 5e-07, 2.5e-07, 1.0000000000000001e-07, 7.500000000000001e-08, 5.0000000000000004e-08, 2.5000000000000002e-08, 1e-08, 7.500000000000001e-09, 5e-09, 2.5e-09]
calbirate: alpha
[array([-6., -6.])]
45000
45000
45000
45000
45000
45000
45000
45000
45000
60000
45000
60000
45000
60000
45000
60000
iteration 0, internal loss=-3241450.42473, time=0.00489711761475
iteration 500, internal loss=-63511.9208008, time=2.50374889374
iteration 1000, internal loss=-63385.1861684, time=2.46901082993
iteration 1500, internal loss=-63409.252271, time=2.60503911972
iteration 2000, internal loss=-63438.6560387, time=2.73696088791
iteration 2500, internal loss=-63415.3516237, time=6.88984584808


KeyboardInterrupt: 

In [10]:
hyp_lr_range

[7.499999999999999e-06]

In [ ]:
#DrMad
cnter = 12
ranges = []
for k in xrange(-9, -3):
    for j in xrange(1,5):
        ranges.append(10**(k)*(j*25))
ranges.sort(reverse=True)
print ranges
hyp_lr_range = [ranges[cnter]] 
print hyp_lr_range

print 'calbirate: alpha'
found = False
for h in [  [np.array([-6.0,-6.0])], 
             [np.array([6.0, 6.0])]]:
    
    print h
    while True:

            
            
            alphas.set_value(h[0])



            score = drmad_optimize(partial(cv_tc, k =4,  batch_size=25), model_build, optimizer,10,25, 5000,  X_train, Y_train,
                                   [alphas] , lr, 
                   lr=hyp_lr_range, verbose=1000 ).history[-1][1]
            if lr.eval()>1 or np.isnan(lr.eval()) or np.isnan(np.sum(alphas.eval())) or \
             max(abs(alphas.eval()))>15 or \
            np.isnan(score) or np.isinf(score):
                print 'BAD'
                cnter+=1
                hyp_lr_range = [ranges[cnter]] 
                print 'new range', hyp_lr_range
            else:
                
                break
            
print 'final', hyp_lr_range            
        
        
        


[0.01, 0.007500000000000001, 0.005, 0.0025, 0.001, 0.00075, 0.0005, 0.00025, 9.999999999999999e-05, 7.5e-05, 4.9999999999999996e-05, 2.4999999999999998e-05, 9.999999999999999e-06, 7.499999999999999e-06, 4.9999999999999996e-06, 2.4999999999999998e-06, 1e-06, 7.5e-07, 5e-07, 2.5e-07, 1.0000000000000001e-07, 7.500000000000001e-08, 5.0000000000000004e-08, 2.5000000000000002e-08]
[9.999999999999999e-06]
calbirate: alpha
[array([-6., -6.])]


In [14]:
hyp_lr_range

[9.999999999999999e-06]

In [ ]:
#HOAG
cnter = 4
ranges = []
for k in xrange(-8, -4):
    for j in xrange(1,5):
        ranges.append(10**(k)*(j*25))
ranges.sort(reverse=True)
print ranges
hyp_lr_range = [ranges[cnter]] 

print 'calbirate: alpha'
found = False
for h in [  [np.array([-6.0,-6.0])],
             [np.array([6.0, 6.0])]]:
    
    print h
    while True:

            
            
            alphas.set_value(h[0])



            score = hoag_optimize(partial(cv_tc, k =4,  batch_size=25), model_build, optimizer,10,25, 500, 5000,  X_train, Y_train,
                                   [alphas] , 
                   lr=hyp_lr_range, verbose=-1,  internal_optimize_learning_rate=10**(-10),
            ).history[-1][1]
            if lr.eval()>1 or np.isnan(lr.eval()) or np.isnan(np.sum(alphas.eval())) or \
             max(abs(alphas.eval()))>15 or \
            np.isnan(score) or np.isinf(score):
                print 'BAD'
                cnter+=1
                hyp_lr_range = [ranges[cnter]] 
                print 'new range', hyp_lr_range
            else:
                
                break
            
print 'final', hyp_lr_range            
        
        
        


[0.001, 0.00075, 0.0005, 0.00025, 9.999999999999999e-05, 7.5e-05, 4.9999999999999996e-05, 2.4999999999999998e-05, 9.999999999999999e-06, 7.499999999999999e-06, 4.9999999999999996e-06, 2.4999999999999998e-06, 1e-06, 7.5e-07, 5e-07, 2.5e-07]
calbirate: alpha
[array([-6., -6.])]
bad internal learning rate nan None nan
updating learning rate 1e-11
updating learning rate 1e-11
updating learning rate 1e-11
updating learning rate 1e-11
bad internal learning rate nan nan nan
updating learning rate 1e-12
updating learning rate 1e-12
updating learning rate 1e-12
updating learning rate 1e-12
bad internal learning rate nan None nan
updating learning rate 1e-13
updating learning rate 1e-13
updating learning rate 1e-13
updating learning rate 1e-13
bad internal learning rate nan nan nan
updating learning rate 1e-14
updating learning rate 1e-14
updating learning rate 1e-14
updating learning rate 1e-14
bad internal learning rate nan None nan
updating learning rate 1e-15
updating learning rate 1e-15
upd

In [12]:
#DrMad-var
cnter = 16
ranges = []
for k in xrange(-9, -3):
    for j in xrange(1,5):
        ranges.append(10**(k)*(j*25))
ranges.sort(reverse=True)
print ranges
hyp_lr_range = [ranges[cnter]] 
print hyp_lr_range
print 'calbirate: alpha'
found = False
for h in [  [np.array([-6.0]*param_num)],
             [np.array([6.0]*param_num)]]:
    
    print h
    while True:

            
            
            var_alphas.set_value(h[0])



            score = drmad_optimize(partial(simple_tc, batch_size=25), var_model_build, optimizer,10,25, 5000,  X_train, Y_train,
                                   [var_alphas] , lr, 
                   lr=hyp_lr_range, verbose=1000
            ).history[-1][1]
            if lr.eval()>1 or np.isnan(lr.eval()) or np.isnan(np.sum(var_alphas.eval())) or \
             max(abs(var_alphas.eval()))>15 or \
            np.isnan(score) or np.isinf(score):
                print 'BAD'
                cnter+=1
                hyp_lr_range = [ranges[cnter]] 
                print 'new range', hyp_lr_range
            else:
                
                break
            
print 'final', hyp_lr_range            
        
        
        


[0.01, 0.007500000000000001, 0.005, 0.0025, 0.001, 0.00075, 0.0005, 0.00025, 9.999999999999999e-05, 7.5e-05, 4.9999999999999996e-05, 2.4999999999999998e-05, 9.999999999999999e-06, 7.499999999999999e-06, 4.9999999999999996e-06, 2.4999999999999998e-06, 1e-06, 7.5e-07, 5e-07, 2.5e-07, 1.0000000000000001e-07, 7.500000000000001e-08, 5.0000000000000004e-08, 2.5000000000000002e-08]
[1e-06]
calbirate: alpha
[array([-6., -6., -6., ..., -6., -6., -6.])]
trial 0
iteration 0, internal loss=-10630388.6711 time=0.00513911247253
iteration 1000, internal loss=-8909577.21513 time=5.16849303246
iteration 2000, internal loss=-8909007.07235 time=5.17952299118
iteration 3000, internal loss=-8909277.80801 time=5.16933202744
iteration 4000, internal loss=-8907367.82361 time=5.17241597176
validation score:  -8908367.96809
reverse-iteration 0, gradients:[[ 2979.95801286  2979.98363824  2979.95800431 ...,  3034.94478956
   3030.66821901  3037.95489358]] time:0.00199699401855
max abs grad 3101.80738358
reverse-i

KeyboardInterrupt: 

In [ ]:
#DrMad-var additional correction
cnter = 10
ranges = []
for k in xrange(-9,     -5):
    for j in xrange(1,5):
        ranges.append(10**(k)*(j*25))
ranges.sort(reverse=True)
print ranges
hyp_lr_range = [ranges[cnter]] 

print 'calbirate: alpha'
found = False
#for h in [  [np.array([-6.0]*param_num)],
#             [np.array([6.0]*param_num)]]:
#print h
while True:
            h = []
            for _ in xrange(param_num):
                h.append(np.random.uniform(low=-6, high=6))
            
            h = np.array(h)
            print h 
            var_alphas.set_value(h)



            score = drmad_optimize(partial(simple_tc, batch_size=25), var_model_build, optimizer,5,25, 500,  X_train, Y_train,
                                   [var_alphas] , lr, 
                   lr=hyp_lr_range, verbose=2500
            ).history[-1][1]
            if lr.eval()>1 or np.isnan(lr.eval()) or np.isnan(np.sum(alphas.eval())) or \
             max(abs(alphas.eval()))>15 or \
            np.isnan(score) or np.isinf(score):
                print 'BAD'
                cnter+=1
                hyp_lr_range = [ranges[cnter]] 
                print 'new range', hyp_lr_range
            else:
                
                break
            
print 'final', hyp_lr_range            
        
        
        


In [ ]:
original, bests = [], []
for _ in xrange(1):
    print 'attemp'
    
    hyp_lr_range = [1.0] 
   
    alphas_value =choicer()
    print alphas_value
    
    alphas.set_value(alphas_value)

    bests.append(greed_optimize(partial(cv_tc, k =4,  batch_size=75), model_build, optimizer,5, 10**3,  X_train, Y_train,  [alphas] , 
               lr=hyp_lr_range, verbose=100, limits=[(-10, 10) ]
    ))
    
    print 'final'
    print bests[-1].history[-1][-1]
    print lr.eval(), alphas.eval()

In [ ]:
print param_num
h = []
for _ in xrange(param_num):
    h.append(np.random.uniform(low=-6, high=6))
            
h = np.array(h)
print h 
var_alphas.set_value(h)

score = drmad_optimize(partial(simple_tc, batch_size=25), var_model_build, optimizer,2,25, 2500,  X_train, Y_train,
                                   [var_alphas] , lr, 
                   lr=[2.5*10**(-6)], verbose=500
            )

In [ ]:
#VAR_Greed
cnter = 0
ranges = []
for k in xrange(-8, -3):
    for j in xrange(1,5):
        ranges.append(10**(k)*(j*25))
ranges.sort(reverse=True)
print ranges
cnter=  5
hyp_lr_range = [ranges[cnter]] 
print hyp_lr_range
print 'calbirate: alpha'
found = False
for h in [  [np.array([-6.0]*param_num)],
             [np.array([6.0]*param_num)]]:
    
    print h
    while True:

            
            
            var_alphas.set_value(h[0])



            score = greed_optimize(partial(simple_tc,  batch_size=25), var_model_build,
                                   optimizer,5,25, 2500,  X_train, Y_train,  [var_alphas] , 
                   lr=hyp_lr_range, verbose=1000
            ).history[-1][1]
            if lr.eval()>1 or np.isnan(lr.eval()) or np.isnan(np.sum(var_alphas.eval())) or \
             max(abs(var_alphas.eval()))>15 or \
            np.isnan(score) or np.isinf(score):
                print 'BAD'
                cnter+=1
                hyp_lr_range = [ranges[cnter]] 
                print 'new range', hyp_lr_range
            else:
                
                break
            
print 'final', hyp_lr_range            
        
        
        
 

[0.01, 0.007500000000000001, 0.005, 0.0025, 0.001, 0.00075, 0.0005, 0.00025, 9.999999999999999e-05, 7.5e-05, 4.9999999999999996e-05, 2.4999999999999998e-05, 9.999999999999999e-06, 7.499999999999999e-06, 4.9999999999999996e-06, 2.4999999999999998e-06, 1e-06, 7.5e-07, 5e-07, 2.5e-07]
[0.00075]
calbirate: alpha
[array([-6., -6., -6., ..., -6., -6., -6.])]


In [ ]:
#VAR_hoag
cnter = 2
ranges = []
for k in xrange(-8, -3):
    for j in xrange(1,5):
        ranges.append(10**(k)*(j*25))
ranges.sort(reverse=True)
print ranges
hyp_lr_range = [ranges[cnter]] 

print 'calbirate: alpha'
found = False
for h in [  [np.array([-6.0]*param_num)],
             [np.array([6.0]*param_num)]]:
    
    print h
    while True:

            
            
            var_alphas.set_value(h[0])



            score = hoag_optimize(partial(simple_tc,  batch_size=25), var_model_build, optimizer,10,25,500, 5000,  X_train, Y_train,
                                   [var_alphas] , 
                   lr=hyp_lr_range, verbose=500,  internal_optimize_learning_rate=10**(-10),
            ).history[-1][1]
            if lr.eval()>1 or np.isnan(lr.eval()) or np.isnan(np.sum(var_alphas.eval())) or \
             max(abs(var_alphas.eval()))>15 or \
            np.isnan(score) or np.isinf(score):
                print 'BAD'
                cnter+=1
                hyp_lr_range = [ranges[cnter]] 
                print 'new range', hyp_lr_range
            else:
                
                break
            
print 'final', hyp_lr_range            
        
        
        


[0.01, 0.007500000000000001, 0.005, 0.0025, 0.001, 0.00075, 0.0005, 0.00025, 9.999999999999999e-05, 7.5e-05, 4.9999999999999996e-05, 2.4999999999999998e-05, 9.999999999999999e-06, 7.499999999999999e-06, 4.9999999999999996e-06, 2.4999999999999998e-06, 1e-06, 7.5e-07, 5e-07, 2.5e-07]
calbirate: alpha
[array([-6., -6., -6., ..., -6., -6., -6.])]
trial  0
iteration 0, internal loss=-10640407.4063
iteration 500, internal loss=-8911445.85713
iteration 1000, internal loss=-8908114.73816
iteration 1500, internal loss=-8908636.8158
iteration 2000, internal loss=-8909694.88247
iteration 2500, internal loss=-8906962.02024


In [7]:
hyp_lr_range

[0.005]

In [ ]:
import theano.tensor as T

training_procedure = cv_tc( model_build,  optimizer, X_train, Y_train, k=4, validation_part=0.25,  batch_size=75 )
X = T.matrix()
result = []
import cPickle
with open('results_cv_greed.pckl','rb') as inp:
    bests = cPickle.load(inp)
for b in bests:
    for i in xrange(len(b.history)):
        training_procedure.models[0].respawn()
        
        alphas.set_value(b.history[i][0][0])
        lr.set_value(0.01)
        
    
        for _ in xrange(100):
            training_procedure.do_train()
        print i, training_procedure.do_validation()
        b.history[i] = [b.history[i][0], training_procedure.do_validation()]
    """
    bests.append( random_optimize(partial(cv_tc, k =4,  batch_size=75), model_build, optimizer, 50, 10, X_train, Y_train,  [alphas, lr] ,
     [alpha_ranges, lr_ranges], verbose=100))
    X = T.matrix()    
    model = model_build(dataset_size=100)
    
    
    """
#print np.mean(result)
#print np.std(result)
import cPickle
with open('results_cv_greed.pckl','wb') as out:
    cPickle.dump(bests, out)

In [ ]:
best_values = []
for b in bests:
    history = []
    for i in xrange(0, len(b.history)):
        
        best_value = max([h[1] for h in b.history[:i+1]])
        
        history.append(best_value)
        if np.isnan(history[-1]):
            continue
        best_values.append(history[-1])
print np.mean(best_values)

In [ ]:
plt.matshow(matrix)
for i in bests:
    i = i.best_values[0]
    i = np.log10(np.exp(i)**2)
    if i[0]>10 or i[1] >10:
        continue
    
    plt.scatter(i[0]+np.random.randn(1)*0.1,i[1]+np.random.randn(1)*0.1, c='w')

In [ ]:
for i in bests:
    print np.array(i.best_values[0])**2

In [6]:
#DrMad
cnter = 12
ranges = []
for k in xrange(-9, -3):
    for j in xrange(1,5):
        ranges.append(10**(k)*(j*25))
ranges.sort(reverse=True)
print ranges
hyp_lr_range = [ranges[cnter]] 
print hyp_lr_range

print 'calbirate: alpha'
found = False
for h in [  [np.array([-6.59,5.63])]]:
    
    print h
    while True:

            
            
            alphas.set_value(h[0])



            score = drmad_optimize(partial(cv_tc, k =4,  batch_size=25), model_build, optimizer,10,25, 5000,  X_train, Y_train,
                                   [alphas] , lr, 
                   lr=hyp_lr_range, verbose=1000 ).history[-1][1]
            if lr.eval()>1 or np.isnan(lr.eval()) or np.isnan(np.sum(alphas.eval())) or \
             max(abs(alphas.eval()))>15 or \
            np.isnan(score) or np.isinf(score):
                print 'BAD'
                cnter+=1
                hyp_lr_range = [ranges[cnter]] 
                print 'new range', hyp_lr_range
            else:
                
                break
            
print 'final', hyp_lr_range            
        
        
        


[0.01, 0.007500000000000001, 0.005, 0.0025, 0.001, 0.00075, 0.0005, 0.00025, 9.999999999999999e-05, 7.5e-05, 4.9999999999999996e-05, 2.4999999999999998e-05, 9.999999999999999e-06, 7.499999999999999e-06, 4.9999999999999996e-06, 2.4999999999999998e-06, 1e-06, 7.5e-07, 5e-07, 2.5e-07, 1.0000000000000001e-07, 7.500000000000001e-08, 5.0000000000000004e-08, 2.5000000000000002e-08]
[9.999999999999999e-06]
calbirate: alpha
[array([-6.59,  5.63])]
trial 0
iteration 0, internal loss=-6336274.69145 time=0.00168085098267
iteration 1000, internal loss=-1053007.17248 time=1.42700695992
iteration 2000, internal loss=-1052848.4511 time=1.42972898483
iteration 3000, internal loss=-1052997.01786 time=1.42304611206
iteration 4000, internal loss=-1052867.02597 time=1.43135690689
validation score:  -1052950.19521
reverse-iteration 0, gradients:[[-5944.02432941   -10.99993813]] time:0.0141620635986
max abs grad 5944.02432941
reverse-iteration 1000, gradients:[[ -1.35819891e+04  -1.09999377e+01]] time:3.0077

KeyboardInterrupt: 